# `Modelagem de problema de estacionamento de avioes num aeroporto utilizando OR-Tools`


Nesse notebook iremos realizar a modelagem e utilizaçao da ferramenta OR-tools para solucionar um problema de logistica de estacionamento de avioes num aeroporto desconsiderando inicialmente o tempo envolvido no problema.

In [1]:
!pip install ortools==9.7.2996 pandas==2.1.0

ERROR: Could not find a version that satisfies the requirement ortools==9.7.2996 (from versions: 9.8.3296, 9.9.3963, 9.10.4067)
ERROR: No matching distribution found for ortools==9.7.2996

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#importe do modelo responsavel pela adiçao de restriçoes no nosso problema
from ortools.sat.python import cp_model

ModuleNotFoundError: No module named 'ortools'

## `Definiçao e construçao dos objetos estacionamento e aviao`

In [ ]:
class Estacionamento:
  def __init__(self, k, total_de_avioes, grande, modelo):
    self.grande = grande
    self.variavel = modelo.NewIntVar(0, total_de_avioes, f'estacionamento_{k}')
    self.k = k
    self.vizinhos = []
    self.recebe_aviao_grande = modelo.NewBoolVar(f'recebe_aviao_grande_{k}')
    if not self.grande:
      modelo.Add(self.recebe_aviao_grande == 0)

In [ ]:
class Aviao:
  def __init__(self, k, grande):
    self.k = k
    self.grande = grande

## `Funçao responsavel por gerar o resultado e o status do modelo criado`

In [ ]:
def resolve(solucionador, modelo, estacionamentos, avioes):
  status = solucionador.Solve(modelo)
  print(solucionador.StatusName(status))
  if status == cp_model.INFEASIBLE:
    print("Sem solucao")
    return
  for estacionamento in estacionamentos:
    variavel = estacionamento.variavel
    valor = solucionador.Value(variavel)
    if valor == 0:
      print(f"{variavel} sem aviao")
    else:
      aviao = avioes[valor - 1]
      print(f"{variavel} tem aviao {valor} grande={aviao.grande}")

## `Funçao que nao permite que o modelo selecione um mesmo aviao para ocupar dois ou mais estacionamentos`

In [ ]:
def avioes_distintos(estacionamentos, modelo):
  variaveis = [estacionamento.variavel for estacionamento in estacionamentos]
  modelo.AddAllDifferent(variaveis)

## `Funçao que incrementa a restriçao (constrain) de que todo aviao deve pousar`

In [ ]:
def todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo):
  variaveis = {}
  for i in range(1, total_de_avioes + 1):
    for j, estacionamento in enumerate(estacionamentos):
      aviao_i_em_j = modelo.NewBoolVar(f'aviao_{i}_em{j}')
      modelo.Add(estacionamento.variavel == i).OnlyEnforceIf(aviao_i_em_j)
      modelo.Add(estacionamento.variavel != i).OnlyEnforceIf(aviao_i_em_j.Not())
      variaveis[(i, j)] = aviao_i_em_j

  for i in range(1, total_de_avioes + 1):
    modelo.AddExactlyOne([variaveis[(i,j)] for j in range(len(estacionamentos))])

## `Funçao que limita avioes grandes de estacionarem apenas em estacionamentos grandes`

In [ ]:
# Agora queremos definir que um aviao grande so pode ser hospedado em um aviao grande

def limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes):
  avioes_grandes = [aviao for aviao in avioes if aviao.grande]
  for estacionamento in estacionamentos:
    # if estacionamento.grande:
    for aviao in avioes_grandes:
      modelo.Add(estacionamento.variavel != aviao.k).OnlyEnforceIf(estacionamento.recebe_aviao_grande.Not())
    # else:
      # nao aceita!!
      # REMOVER AVIOES GRANDES!
      # remover_do_estacionamento(modelo, estacionamento.variavel, avioes_grandes)

## `Funçao que adiciona a restriçao de que se um estacionamento for grande e tiver um aviao grande em algum vizinho ele nao podera receber um aviao grande`

In [ ]:
def limita_vizinhos(modelo, estacionamentos, avioes):
  for estacionamento in estacionamentos:
    if not estacionamento.grande:
      continue
    for vizinho in estacionamento.vizinhos:
      if vizinho.grande:
        # se o vizinho tem um aviao grande => eu nao posso receber aviao grande
        modelo.Add(estacionamento.recebe_aviao_grande == 0).OnlyEnforceIf(vizinho.recebe_aviao_grande)


## `Segue abaixo exemplos testando o modelo para demonstrar sua validade`

In [ ]:
#Exemplo 1:

avioes = [Aviao(1, True),
          Aviao(2, False),
          Aviao(3, True)]
modelo = cp_model.CpModel()
total_de_avioes = len(avioes)

estacionamentos = [Estacionamento(1, total_de_avioes, False, modelo),
                   Estacionamento(2, total_de_avioes, False, modelo),
                   Estacionamento(3, total_de_avioes, True, modelo),
                   Estacionamento(4, total_de_avioes, True, modelo)]
estacionamentos[2].vizinhos = [estacionamentos[3]]

avioes_distintos(estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo)
limita_vizinhos(modelo, estacionamentos, avioes)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, estacionamentos, avioes)

INFEASIBLE
Sem solucao


In [ ]:
#Exemplo 2
avioes = [Aviao(1, True),
          Aviao(2, False),
          Aviao(3, True)]
modelo = cp_model.CpModel()
total_de_avioes = len(avioes)

estacionamentos = [Estacionamento(1, total_de_avioes, False, modelo),
                   Estacionamento(2, total_de_avioes, False, modelo),
                   Estacionamento(3, total_de_avioes, True, modelo),
                   Estacionamento(4, total_de_avioes, True, modelo)]
estacionamentos[2].vizinhos = [estacionamentos[0]]

avioes_distintos(estacionamentos, modelo)
todo_aviao_tem_que_estacionar(total_de_avioes, estacionamentos, modelo)
limita_vizinhos(modelo, estacionamentos, avioes)
limita_aviao_grande_para_estacionamento_grande(modelo, estacionamentos, avioes)

solucionador = cp_model.CpSolver()
resolve(solucionador, modelo, estacionamentos, avioes)

OPTIMAL
estacionamento_1 tem aviao 2 grande=False
estacionamento_2 sem aviao
estacionamento_3 tem aviao 3 grande=True
estacionamento_4 tem aviao 1 grande=True
